In [59]:
import os
os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

import pandas as pd 
import numpy as np 
import plotly.express as px 

from mirrorverse.utils import read_data_w_cache

In [ ]:
model = '3_3_3'
data = read_data_w_cache(
    f'select * from chinook_depth_inference_{model} where not _train'
)
print(data.shape)
data.head()

In [ ]:
data.groupby(['depth_bin'])[['_selected', 'probability']].mean()

In [ ]:
data['log_likelihood'] = np.log(data['probability'])
data[data['_selected'] & ~data['_train']].groupby(['_individual'])['log_likelihood'].mean().reset_index()['log_likelihood'].mean()

In [ ]:
data['null_odds'] = 1.0
data['sum_null_odds'] = data.groupby(['_individual', '_decision'])['null_odds'].transform('sum')
data['null_probability'] = data['null_odds'] / data['sum_null_odds']
data.groupby(['depth_bin'])[['_selected', 'probability', 'null_probability']].mean()

In [ ]:
data['null_log_likelihood'] = np.log(data['null_probability'])
data[data['_selected'] & ~data['_train']].groupby(['_individual'])['null_log_likelihood'].mean().reset_index()['null_log_likelihood'].mean()

In [ ]:
data[data['_selected'] & ~data['_train']][['probability', 'null_probability']].mean()

In [ ]:
color_discrete_map = {
    0.1: "#1b9e77",  # Green
    0.2: "#d95f02",  # Orange
    0.3: "#7570b3",  # Purple
    0.4: "#e7298a",  # Pink
    0.5: "#66a61e",  # Olive Green
    0.6: "#e6ab02",  # Yellow-Orange
    0.7: "#a6761d",  # Brown
    0.8: "#666666",  # Gray
    0.9: "#1f78b4",  # Blue
    1.0: "#a6cee3",  # Light Blue
}

data['orbit_radians'] = np.arctan2(data['sin_orbit'], data['cos_orbit'])
data['binned_orbit_radians'] = round(data['orbit_radians'] * 10) / (10)
df = data.groupby(['binned_orbit_radians', 'n_depth_bin'])['probability'].mean().reset_index()
px.line(
    df.sort_values('binned_orbit_radians'), x='binned_orbit_radians', y='probability', color='n_depth_bin', 
    color_discrete_map=color_discrete_map, title=f'model - {model}'

)

In [ ]:
data['binned_n_mixed_layer_thickness'] = round(data['n_mixed_layer_thickness'] * 10) / (10)
df = data.groupby(['binned_n_mixed_layer_thickness', 'n_depth_bin'])['probability'].mean().reset_index()
px.line(
    df.sort_values('binned_n_mixed_layer_thickness'), x='binned_n_mixed_layer_thickness', y='probability', color='n_depth_bin', 
    color_discrete_map=color_discrete_map, title=f'model - {model}'
)

In [ ]:
data['binned_n_salinity'] = round(data['n_salinity'] * 10) / (10)
df = data.groupby(['binned_n_salinity', 'n_depth_bin'])['probability'].mean().reset_index()
px.line(
    df.sort_values('binned_n_salinity'), x='binned_n_salinity', y='probability', color='n_depth_bin', 
    color_discrete_map=color_discrete_map, title=f'model - {model}'
)

In [ ]:
data['sun_radians'] = np.arctan2(data['sin_sun'], data['cos_sun'])
data['binned_sun_radians'] = round(data['sun_radians'] * 10) / (10)
df = data.groupby(['binned_sun_radians', 'n_depth_bin'])['probability'].mean().reset_index()
px.line(
    df.sort_values('binned_sun_radians'), x='binned_sun_radians', y='probability', color='n_depth_bin', 
    color_discrete_map=color_discrete_map, title=f'model - {model}'

)